How we one hot encode game state into observation state with one hot encoding.

Because TFT is a game with a high amount of categorical data, we need to one hot encode so that it can be processed by our machine for reinforcement learning.

High Cardinality Categorical Data

There are 100s of items and champions in TFT that need to be encoded. Each champion can hold 3 items and
each player can have up to 9 champions on bench and board, so the number of combination grows extremely quickly.

This is how I've tried to encode this data.

In [2]:
import json
import sys
import random
from sklearn.preprocessing import OneHotEncoder
import numpy as np
np.set_printoptions(threshold=sys.maxsize)


champions_data = json.load(open('../tft_static_data/set5patch1115/champions.json'))
items_data = json.load(open('../tft_static_data/set5patch1115/items.json'))


champion_ids = [c['championId'] for c in champions_data]
item_ids = [str(i['id']) for i in items_data]

print(champion_ids)
print("\n")
print(item_ids)

['TFT5_Aatrox', 'TFT5_Akshan', 'TFT5_Aphelios', 'TFT5_Ashe', 'TFT5_Brand', 'TFT5_Diana', 'TFT5_Draven', 'TFT5_Fiddlesticks', 'TFT5_Galio', 'TFT5_Garen', 'TFT5_Gragas', 'TFT5_Gwen', 'TFT5_Hecarim', 'TFT5_Heimerdinger', 'TFT5_Ivern', 'TFT5_Irelia', 'TFT5_Jax', 'TFT5_Kalista', 'TFT5_Karma', 'TFT5_Kayle', 'TFT5_Kennen', 'TFT5_Khazix', 'TFT5_Kled', 'TFT5_LeeSin', 'TFT5_Leona', 'TFT5_Lucian', 'TFT5_Lulu', 'TFT5_Lux', 'TFT5_MissFortune', 'TFT5_Nautilus', 'TFT5_Nidalee', 'TFT5_Nocturne', 'TFT5_Nunu', 'TFT5_Olaf', 'TFT5_Poppy', 'TFT5_Pyke', 'TFT5_Rakan', 'TFT5_Rell', 'TFT5_Riven', 'TFT5_Sejuani', 'TFT5_Senna', 'TFT5_Sett', 'TFT5_Soraka', 'TFT5_Syndra', 'TFT5_Teemo', 'TFT5_Thresh', 'TFT5_Tristana', 'TFT5_Udyr', 'TFT5_Varus', 'TFT5_Vayne', 'TFT5_Velkoz', 'TFT5_Viego', 'TFT5_Vladimir', 'TFT5_Volibear', 'TFT5_Yasuo', 'TFT5_Ziggs', 'TFT5_Zyra']


['1', '2', '3', '4', '5', '6', '7', '8', '9', '11', '12', '13', '14', '15', '16', '17', '18', '19', '22', '23', '24', '25', '26', '27', '28', '29', '33', '

In [3]:
# Mock out what the game state we want to encode will look like

# A board will consists of up to 9 champions, each with 3 item slots and their level.
# A champion is represented by a array of size 4 with [CHAMPION_ID, ITEM1, ITEM2, ITEM3, CHAMPION_LEVEL]
board = [
    [
        random.choice(champion_ids), 
        random.choice(item_ids), 
        random.choice(item_ids), 
        random.choice(item_ids), 
        random.randint(1,3)
    ] for i in range(8)
]
board.append(['None',0,0,0,0]) # add an unoccupied board slot

bench = [
    [
        random.choice(champion_ids), 
        random.choice(item_ids), 
        random.choice(item_ids), 
        random.choice(item_ids), 
        random.randint(1,3)
    ] for i in range(8)
]
bench.append(['None',0,0,0,0])

# SHOP just has the champion names
shop = [random.choice(champion_ids) for i in range(5)] 

print("BOARD:", board, "\n\nBENCH:", bench, "\n\nSHOP:", shop)


BOARD: [['TFT5_Heimerdinger', '2066', '2099', '66', 1], ['TFT5_Galio', '26', '2099', '2016', 3], ['TFT5_Teemo', '78', '47', '9', 3], ['TFT5_Kennen', '2013', '79', '1128', 3], ['TFT5_Nunu', '1191', '57', '1193', 1], ['TFT5_Brand', '67', '9', '1189', 2], ['TFT5_Sejuani', '25', '59', '38', 1], ['TFT5_Nocturne', '1190', '2079', '77', 1], ['None', 0, 0, 0, 0]] 

BENCH: [['TFT5_Syndra', '2033', '57', '2012', 3], ['TFT5_Irelia', '1168', '17', '2016', 1], ['TFT5_Irelia', '46', '2079', '1128', 1], ['TFT5_Kayle', '1158', '6', '34', 1], ['TFT5_Nidalee', '34', '16', '17', 1], ['TFT5_Aphelios', '2049', '36', '12', 1], ['TFT5_Rakan', '1196', '2029', '2046', 3], ['TFT5_Olaf', '17', '38', '88', 1], ['None', 0, 0, 0, 0]] 

SHOP: ['TFT5_Kalista', 'TFT5_Galio', 'TFT5_Ziggs', 'TFT5_Khazix', 'TFT5_Olaf']


In [4]:
# Now we need to represent this game state as a 1-d vector. 
# Note that since we work with np arrays, everything is converted to 
# a string
flattened_game_state = np.hstack(shop+board+bench)
print(flattened_game_state)
# Flattened game state should be 95 items long (Shop=5, Board=45, Bench=45)
assert len(flattened_game_state) == 5 + 9*5 + 9*5  

['TFT5_Kalista' 'TFT5_Galio' 'TFT5_Ziggs' 'TFT5_Khazix' 'TFT5_Olaf'
 'TFT5_Heimerdinger' '2066' '2099' '66' '1' 'TFT5_Galio' '26' '2099'
 '2016' '3' 'TFT5_Teemo' '78' '47' '9' '3' 'TFT5_Kennen' '2013' '79'
 '1128' '3' 'TFT5_Nunu' '1191' '57' '1193' '1' 'TFT5_Brand' '67' '9'
 '1189' '2' 'TFT5_Sejuani' '25' '59' '38' '1' 'TFT5_Nocturne' '1190'
 '2079' '77' '1' 'None' '0' '0' '0' '0' 'TFT5_Syndra' '2033' '57' '2012'
 '3' 'TFT5_Irelia' '1168' '17' '2016' '1' 'TFT5_Irelia' '46' '2079' '1128'
 '1' 'TFT5_Kayle' '1158' '6' '34' '1' 'TFT5_Nidalee' '34' '16' '17' '1'
 'TFT5_Aphelios' '2049' '36' '12' '1' 'TFT5_Rakan' '1196' '2029' '2046'
 '3' 'TFT5_Olaf' '17' '38' '88' '1' 'None' '0' '0' '0' '0']


In [7]:
# We now need to write a one-hot encoder that can hanlde all possible permutations of game state. The one-hot-encoder
# must be able to encode all possible champion and item combinations.

# list : categories[i] holds the categories expected in the ith column. 
# The passed categories should not mix strings and numeric values within 
# a single feature, and should be sorted in case of numeric values.

categories = []
# First five categories is the shop, which can hold CHAMPION_IDs
categories += [champion_ids] * 5
# Next is the bench and board 

for i in range(18):
    categories += [['None'] + champion_ids] # the champion name
    categories += [['0'] + item_ids] * 3 # 3 items champ can hold
    categories.append(["0", "1","2","3"]) # champ levels

game_state_encoder = OneHotEncoder(
    categories=categories,
    # drop=True <- We may want to drop first column due to collinearity.
)

game_state_encoder._fit = True

# game_state_encoder.fit([flattened_game_state])


# Test one hot encoding game state works!
one_hot_encoded_state = game_state_encoder.transform([flattened_game_state]).toarray()[0]
print("Length of onehotencoded state:", len(one_hot_encoded_state), "\n")

# Test if reverse decoding game state works:
game_state_encoder.inverse_transform([one_hot_encoded_state])



NotFittedError: This OneHotEncoder instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [6]:
# We now have our encoder: game_state_encoder
# We might want to do a few things. We can pickle it so we can load it
# from anywhere for use
from joblib import dump, load
import os.path

dump(game_state_encoder, './game_state_encoder.joblib') 
encoder = load('./game_state_encoder.joblib') 